In [207]:
import numpy as np

def hopfield_prng(num_outputs, N=100, L=10, b=0.1, delta=0.01):
    """
    Generates pseudo-random numbers using a Hopfield neural network
    Parameters:
    num_outputs: Number of random values to generate
    N: Number of neurons (default=100)
    L: Iterations per output (default=10)
    b: Activation scaling factor (default=0.1)
    delta: Weight perturbation magnitude (default=0.01)
    """
    # Initialize asymmetric weight matrix
    T = np.random.uniform(-1, 1, (N, N))
    np.fill_diagonal(T, np.diag(T) * 2)  # Enhance diagonal
    
    # Initialize neuron states
    V = np.random.uniform(-1, 1, N)
    I = np.zeros(N)  # External input
    
    outputs = []
    
    for _ in range(num_outputs):
        # Weight perturbation phase
        T = T + delta * np.triu(np.ones((N, N))) - delta * np.tril(np.ones((N, N)), -1)
        
        # Neuron activation phase
        for _ in range(L):
            idx = np.random.choice(N, size=N//2 + 1, replace=False)
            U = np.dot(T, V) - I
            V[idx] = np.tanh(b * U[idx])
        
        # Random number extraction
        scaled = np.sum(np.tanh(T @ V)) / (L * N)
        fractional = (1000 * scaled) % 1
        outputs.append(fractional)
    
    return outputs


In [ ]:
def tm_hnn_prng(num_outputs):
    """
    TMHNN PRNG
    """
    # Chaotic system implementation would go here
    # For brevity, showing the bit extraction method:
    random_numbers = []
    for _ in range(num_outputs):
        # Generate chaotic state (simplified)
        state = np.random.uniform(-10, 10)  # Prototype of TM-HNN equations
        
        # IEEE 754 conversion
        float_bytes = np.float32(state).tobytes()
        bits = ''.join(f'{byte:08b}' for byte in float_bytes)[16:32]  # Bits 17-32
        random_numbers.append(int(bits, 2) / 65535.0)  # Normalize to [0,1)
    
    return random_numbers

In [ ]:
def dbhnn_prng(num_outputs, μ=0.8, K=1000):
    """
    DBHNN PRNG
    μ: Internal decay coefficient (0.3, 0.6, 0.8, or 0.95)
    K: Scaling constant
    """
    # Initialize
    x, y = np.random.uniform(-1, 1), np.random.uniform(-1, 1)
    g11, g12, g21, g22 = 2, 2, -2, -2
    outputs = []
    
    for _ in range(num_outputs):
        # Update equations
        x_new = μ * x + g11 * np.sin(x) + g12 * np.sin(y)
        y_new = μ * y + g21 * np.sin(x) + g22 * np.sin(y)
        x, y = x_new, y_new
        
        # Output transformation
        Xm = (x + y) % 1
        prng_output = int(Xm * K) % 256 / 255.0  # 8-bit output
        outputs.append(prng_output)
    
    return outputs


In [210]:
import hashlib

def hopfield_sha_prng(num_outputs):
    """
    My Method : Hopfield + SHA-2 hashing
    """
    # Generate raw sequence with canonical Hopfield
    raw_sequence = hopfield_prng(num_outputs)
    
    # Convert to bytes and hash
    byte_data = b''.join(int(val * 10**9).to_bytes(4, 'big') for val in raw_sequence)
    hashed = hashlib.sha256(byte_data).digest()
    
    # Convert hash to float in [0,1)
    return [int.from_bytes(hashed[i:i+4], 'big') / 2**32 for i in range(0, len(hashed), 4)][:num_outputs]


In [211]:
# Canonical Hopfield
random_nums = hopfield_prng(1)

# TM-HNN variant
tm_random = tm_hnn_prng(1)

# DBHNN variant
dbhnn_random = dbhnn_prng(1, μ=0.95)

# Combined Hopfield+SHA2
secure_random = hopfield_sha_prng(1)

print(random_nums,'\n',tm_random,'\n',dbhnn_random,'\n',secure_random)

[0.06018954505006757] 
 [0.8447241931792172] 
 [0.796078431372549] 
 [0.8823925363831222]
